In [ ]:
import openai
import random
import pandas as pd
from sklearn.metrics import f1_score


In [ ]:
import pandas as pd
import csv
import os
import time
import random
import numpy as np
from sklearn.metrics import f1_score
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# OpenAI API key
OPENAI_API_KEY = "your_api_key_here"

In [ ]:
roles = [
    "Gene interaction analyst",
    "Pathway relationship analyst",
    "Molecular biologist specializing in gene interactions",
    "Gene interaction specialist",
    "Molecular genetics expert",
    "Computational biologist",
    "Gene interaction analyst",
    "Bioinformatics researcher",
    "Molecular genetics expert",
    "Gene interaction analyst",
    "Molecular biologist specializing in gene interactions",
    "Gene interaction specialist",
    "Gene interaction analyst",
    "Molecular biologist specializing in gene interactions",
    "Bioinformatics researcher",
    "Gene interaction specialist",
    "Molecular genetics expert",
    "Computational biologist",
    "Molecular biologist specializing in gene interactions",
    "Gene interaction analyst",
    "Bioinformatics researcher",
    "Gene interaction specialist",
    "Gene interaction analyst",
    "Molecular genetics expert",
    "Gene interaction specialist",
    "Molecular biologist specializing in gene interactions",
    "Molecular genetics expert",
    "Gene interaction analyst",
    "Bioinformatics researcher",
    "Molecular biologist specializing in gene interactions",
    "Gene interaction specialist",
    "Gene interaction analyst",
    "Molecular biologist specializing in gene interactions",
    "Molecular genetics expert",
    "Bioinformatics researcher",
    "Gene interaction specialist",
    "Molecular genetics expert",
    "Computational biologist",
    "Bioinformatics researcher",
    "Gene interaction specialist",
    "Molecular genetics expert",
    "Molecular biologist specializing in gene interactions",
    "Gene interaction analyst",
    "Molecular biologist specializing in gene interactions",
    "Molecular genetics expert",
    "Bioinformatics researcher"
]
aims = [
    "Extract gene relationships from the provided pathway diagram, ensuring accuracy in categorizing interaction types.",
    "Extract and classify gene interactions from the provided pathway diagram.",
    "Distinguish and document gene interactions from the provided pathway diagram.",
    "Analyze gene interactions within the provided diagram, ensuring accurate extraction and categorization.",
    "Extract gene interactions from the pathway diagram, categorizing each interaction accurately.",
    "Extract and document gene interactions from the provided pathway diagram.",
    "Extract and categorize gene interactions from the provided pathway diagram.",
    "Extract gene interactions from the pathway diagram with accuracy and precision.",
    "Extract and classify gene interactions from the provided pathway diagram.",
    "Extract and label gene interactions from the pathway diagram with precision.",
    "Accurately extract and classify gene relationships from the provided pathway diagram.",
    "Extract and categorize gene interactions from the provided diagram with accuracy.",
    "Accurately identify and document gene interactions within the pathway diagram.",
    "Extract gene relationships from the pathway diagram with precise classification.",
    "Analyze and extract gene interactions from the pathway diagram with accuracy.",
    "Extract and classify gene interactions from the provided diagram, ensuring precision.",
    "Identify and extract gene relationships from the pathway diagram with clarity.",
    "Extract and classify gene interactions from the pathway diagram, ensuring clarity.",
    "Identify and document gene relationships from the pathway diagram with accuracy.",
    "Extract and categorize gene interactions from the pathway diagram with precision.",
    "Accurately extract and classify gene relationships from the provided pathway diagram.",
    "Extract and accurately document gene interactions from the pathway diagram.",
    "Accurately extract and classify gene interactions from the pathway diagram.",
    "Extract and classify gene interactions from the pathway diagram, ensuring accuracy.",
    "Accurately extract gene interactions from the provided pathway diagram.",
    "Extract and accurately document gene relationships from the pathway diagram.",
    "Extract and classify gene interactions from the pathway diagram with precision.",
    "Accurately extract and classify gene relationships from the provided pathway diagram.",
    "Extract and categorize gene interactions from the pathway diagram with precision.",
    "Accurately extract gene interactions from the pathway diagram.",
    "Extract and categorize gene interactions from the pathway diagram with precision.",
    "Accurately identify and document gene interactions within the pathway diagram.",
    "Extract and classify gene relationships from the provided pathway diagram.",
    "Identify and categorize gene relationships from the pathway diagram with precision.",
    "Accurately extract gene interactions from the provided pathway diagram.",
    "Accurately extract and categorize gene interactions from the pathway diagram.",
    "Extract and document gene relationships from the pathway diagram with accuracy.",
    "Extract and classify gene interactions from the pathway diagram with precision.",
    "Accurately extract and document gene interactions from the provided pathway diagram.",
    "Extract and classify gene relationships from the pathway diagram with precision.",
    "Accurately document and classify gene relationships from the pathway diagram.",
    "Extract and categorize gene interactions from the pathway diagram with precision.",
    "Accurately extract and document gene interactions within the pathway diagram.",
    "Extract and classify gene relationships from the provided pathway diagram.",
    "Identify and categorize gene relationships from the pathway diagram with precision.",
    "Accurately extract gene interactions from the provided pathway diagram.",
    "Extract and classify gene interactions from the pathway diagram with precision.",
    "Accurately extract and document gene interactions from the provided pathway diagram.",
    "Extract and classify gene relationships from the pathway diagram with precision."
]

instruction = [
    "Symbols in the diagram represent specific interactions: 'Inhibition' is shown by T-bar symbols (----|), with dashed T-bars indicating 'Indirect Inhibition.' 'Activation' is represented by arrow symbols (?), with dashed arrows for 'Indirect Activation.' Arrows point from gene1 (starter) to gene2 (receptor).",
    "Use the diagram's symbols: 'Inhibition' is marked by T-bars (----|), with dashed T-bars for 'Indirect Inhibition.' 'Activation' is denoted by arrows (?), with dashed arrows for 'Indirect Activation.' The arrow direction indicates the flow between gene1 and gene2.",
    "Refer to the symbols in the diagram: T-bars (----|) indicate 'Inhibition,' with dashed T-bars for 'Indirect Inhibition.' Arrows (?) show 'Activation,' with dashed arrows representing 'Indirect Activation.' Arrows indicate the direction of interaction from gene1 to gene2.",
    "Use the provided symbols in the diagram: T-bars (----|) denote 'Inhibition,' with dashed T-bars for 'Indirect Inhibition.' Arrows (?) represent 'Activation,' with dashed arrows for 'Indirect Activation.' The direction of arrows shows the relationship flow from gene1 to gene2.",
    "Identify relationships using the diagram's symbols: T-bars (----|) represent 'Inhibition,' with dashed T-bars indicating 'Indirect Inhibition.' Arrows (?) show 'Activation,' with dashed arrows signifying 'Indirect Activation.' Arrows indicate the flow from gene1 to gene2.",
    "The diagram uses specific symbols: T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition.' Arrows (?) for 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows indicate the direction of the interaction from gene1 to gene2.",
    "Symbols in the diagram represent interactions: T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition.' Arrows (?) for 'Activation,' with dashed arrows for 'Indirect Activation.' The direction of the arrows shows the flow between gene1 and gene2.",
    "Use the symbols provided in the diagram: T-bars (----|) represent 'Inhibition,' with dashed T-bars indicating 'Indirect Inhibition.' Arrows (?) denote 'Activation,' with dashed arrows representing 'Indirect Activation.' Arrows point from gene1 (starter) to gene2 (receptor).",
    "Refer to the diagram's symbols: T-bars (----|) indicate 'Inhibition,' with dashed T-bars representing 'Indirect Inhibition.' Arrows (?) signify 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows indicate the direction from gene1 to gene2.",
    "Use the symbols in the diagram: T-bars (----|) for 'Inhibition,' with dashed T-bars indicating 'Indirect Inhibition.' Arrows (?) represent 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows show the relationship flow from gene1 to gene2.",
    "The pathway diagram uses symbols to show interactions: 'Inhibition' is represented by T-bar (----|), with dashed T-bars indicating 'Indirect Inhibition.' 'Activation' is shown with arrows (?), with dashed arrows indicating 'Indirect Activation.' Arrows illustrate the direction from gene1 to gene2.",
    "Symbols used in the diagram include T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows point from gene1 to gene2 to indicate direction.",
    "Refer to the pathway diagram symbols: 'Inhibition' is shown by T-bars (----|), with dashed T-bars for 'Indirect Inhibition.' 'Activation' is indicated by arrows (?), with dashed arrows for 'Indirect Activation.' The direction of arrows shows the flow from gene1 to gene2.",
    "The pathway diagram uses T-bars (----|) for 'Inhibition,' with dashed T-bars indicating 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows representing 'Indirect Activation.' The arrow's direction indicates the flow from gene1 to gene2.",
    "Symbols in the pathway diagram include T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows representing 'Indirect Activation.' Arrows point from gene1 to gene2.",
    "The diagram uses T-bars (----|) to depict 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) to show 'Activation,' with dashed arrows representing 'Indirect Activation.' The direction of arrows shows interaction flow from gene1 to gene2.",
    "In the pathway diagram, T-bars (----|) indicate 'Inhibition,' with dashed T-bars representing 'Indirect Inhibition,' and arrows (?) signify 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows point from gene1 to gene2.",
    "The diagram uses T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows indicate the direction from gene1 to gene2.",
    "Symbols in the diagram include T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows representing 'Indirect Activation.' Arrows show the direction from gene1 to gene2.",
    "The diagram depicts interactions with T-bars (----|) for 'Inhibition,' with dashed T-bars indicating 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows point from gene1 to gene2.",
    "The pathway diagram illustrates interactions using T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) for 'Activation,' with dashed arrows representing 'Indirect Activation.' The direction of arrows shows the flow from gene1 to gene2.",
    "The diagram uses specific symbols to depict interactions: T-bars (----|) indicate 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) represent 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows indicate the direction from gene1 to gene2.",
    "Refer to the symbols in the pathway diagram: 'Inhibition' is represented by T-bars (----|), with dashed T-bars for 'Indirect Inhibition,' and 'Activation' is shown by arrows (?), with dashed arrows for 'Indirect Activation.' The arrows indicate the relationship direction from gene1 to gene2.",
    "In the diagram, 'Inhibition' is shown by T-bars (----|), with dashed T-bars indicating 'Indirect Inhibition,' and 'Activation' is represented by arrows (?), with dashed arrows for 'Indirect Activation.' Arrows indicate the interaction direction from gene1 to gene2.",
    "Symbols in the diagram represent interactions: T-bars (----|) indicate 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) represent 'Activation,' with dashed arrows for 'Indirect Activation.' The arrows point from gene1 to gene2.",
    "In the diagram, 'Inhibition' is indicated by T-bars (----|), with dashed T-bars for 'Indirect Inhibition,' and 'Activation' is shown by arrows (?), with dashed arrows for 'Indirect Activation.' The arrows illustrate the direction from gene1 to gene2.",
    "The diagram uses T-bars (----|) to indicate 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) to show 'Activation,' with dashed arrows representing 'Indirect Activation.' The direction of arrows points from gene1 to gene2.",
    "In the pathway diagram, 'Inhibition' is indicated by T-bars (----|), with dashed T-bars for 'Indirect Inhibition,' and 'Activation' is shown by arrows (?), with dashed arrows for 'Indirect Activation.' The direction of arrows shows the relationship from gene1 to gene2.",
    "The diagram depicts interactions using T-bars (----|) to indicate 'Inhibition,' with dashed T-bars for 'Indirect Inhibition,' and arrows (?) to show 'Activation,' with dashed arrows representing 'Indirect Activation.' The arrows indicate the direction from gene1 to gene2.",
    "Identify relationships using the diagram's symbols: T-bars (----|) represent 'Inhibition,' with dashed T-bars indicating 'Indirect Inhibition.' Arrows (?) show 'Activation,' with dashed arrows signifying 'Indirect Activation.' Arrows indicate the flow from gene1 to gene2.",
    "The diagram uses specific symbols: T-bars (----|) for 'Inhibition,' with dashed T-bars for 'Indirect Inhibition.' Arrows (?) for 'Activation,' with dashed arrows for 'Indirect Activation.' Arrows indicate the direction of the interaction from gene1 to gene2."
]
descriptions = [
    "Identify each gene interaction, differentiate between 'direct' and 'indirect' interactions, and format them as 'gene1 relationship gene2.'",
    "Differentiate between 'inhibition' and 'activation' as well as between 'direct' and 'indirect' interactions, and format the results as 'gene1 relationship gene2.'",
    "Analyze and extract each interaction, labeling them as 'direct' or 'indirect' and formatting them as 'gene1 relationship gene2.'",
    "Carefully extract each interaction, distinguishing between 'direct' and 'indirect' interactions, and format them as 'gene1 relationship gene2.'",
    "Classify each interaction as either 'direct' or 'indirect' and format it as 'gene1 relationship gene2.'",
    "Extract and classify each gene interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Identify and classify each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Accurately extract and label each interaction as 'direct' or 'indirect,' formatting them as 'gene1 relationship gene2.'",
    "Identify, classify, and format each interaction as 'gene1 relationship gene2.'",
    "Extract and categorize each interaction, distinguishing between 'direct' and 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Identify and differentiate each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract each interaction, classify it as 'direct' or 'indirect,' and format it as 'gene1 relationship gene2.'",
    "Analyze and classify each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract and distinguish each interaction as 'direct' or 'indirect,' formatting them as 'gene1 relationship gene2.'",
    "Identify and distinguish each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract, classify, and distinguish each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Identify and classify each interaction, distinguishing between 'direct' and 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract and differentiate each interaction, classifying them as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract and classify each interaction, determining whether it is 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Analyze and classify each interaction, distinguishing between 'direct' and 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract and differentiate each interaction, classifying it as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract, classify, and differentiate each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Identify each interaction, distinguish between 'direct' and 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Carefully extract each interaction, categorize it as 'direct' or 'indirect,' and format it as 'gene1 relationship gene2.'",
    "Identify and categorize each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Carefully extract each interaction, classify it as 'direct' or 'indirect,' and format it as 'gene1 relationship gene2.'",
    "Extract each interaction, categorize it as 'direct' or 'indirect,' and format it as 'gene1 relationship gene2.'",
    "Identify, classify, and differentiate each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract, classify, and distinguish each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Identify and classify each interaction, distinguishing between 'direct' and 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Extract, classify, and differentiate each interaction as 'direct' or 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Identify each interaction, distinguish between 'direct' and 'indirect,' and format them as 'gene1 relationship gene2.'",
    "Carefully extract each interaction, classify it as 'direct' or 'indirect,' and format it as 'gene1 relationship gene2.'",
    "Carefully extract each interaction, categorize it as 'direct' or 'indirect,' and format it as 'gene1 relationship gene2.'"
]


In [ ]:
def ask_question(prompt, question, temperature=0.9):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": prompt}, {"role": "user", "content": question}],
            temperature=temperature
        )
        return response["choices"][0]["message"]["content"].strip().lower()
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

def calculate_metrics(df):
    relations = ['activation', 'inhibition', 'phosphorylation', 'no information']
    f1_scores = {relation: f1_score(df['relation'] == relation, df['predict_relation'] == relation, zero_division=0) for relation in relations}
    overall_f1 = f1_score(df['relation'], df['predict_relation'], average='micro', zero_division=0)
    return overall_f1, f1_scores

def get_fitness(prompt, question, temperature=0.9, training_path=TRAINING_PATH):
    answers = []
    training_df = pd.read_excel(TRAINING_PATH).sample(frac=1).reset_index(drop=True)
    for _, row in training_df.iterrows():
        gpt4o_answer = ask_question(prompt, question.format(gene1=row['starter'].upper(), gene2=row['receiver'].upper()), temperature)
        answers.append({'starter': row['starter'], 'receiver': row['receiver'], 'relation': row['relation_name'], 'predict_relation': gpt4o_answer})

    answer_df = pd.DataFrame(answers)
    overall_f1, _ = calculate_metrics(answer_df)
    return overall_f1

def select_parents(population, fitness_values):
    probabilities = [fitness / sum(fitness_values) for fitness in fitness_values]
    return random.choices(population, weights=probabilities, k=int(POPULATION_SIZE * SELECTION_PROPORTION))

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1) - 1)
    return (parent1[:point] + parent2[point:], parent2[:point] + parent1[point:])

def mutate(individual):
    mutated = list(individual)
    index = random.randint(0, len(mutated) - 1)
    options = [roles, aims, descriptions, instruction][index]
    mutated[index] = random.choice(options)
    return tuple(mutated)

def generate_prompt(individual):
    return f"As a {individual[0]}, {individual[1]}. {individual[2]} {individual[3]}."

# Genetic algorithm parameters
POPULATION_SIZE = 9
NUM_GENERATIONS = 30
MUTATION_RATE = 0.3
SELECTION_PROPORTION = 0.5

population = [(random.choice(roles), random.choice(aims), random.choice(descriptions), random.choice(instruction)) for _ in range(POPULATION_SIZE)]

iteration_data = []

for generation in range(NUM_GENERATIONS):
    fitness_values = [get_fitness(generate_prompt(ind), "Extract the interaction between {gene1} and {gene2}.") for ind in population]
    new_population = select_parents(population, fitness_values)

    while len(new_population) < POPULATION_SIZE:
        p1, p2 = random.sample(new_population, 2)
        c1, c2 = crossover(p1, p2)
        if random.random() < MUTATION_RATE:
            c1 = mutate(c1)
        if random.random() < MUTATION_RATE:
            c2 = mutate(c2)
        new_population.extend([c1, c2])

    population = sorted(new_population[:POPULATION_SIZE], key=lambda ind: get_fitness(generate_prompt(ind), "Extract the interaction between {gene1} and {gene2}.", temperature=0.9), reverse=True)

    best_f1 = max(fitness_values)
    iteration_data.append({"Iteration": generation + 1, "Best F1 Score": best_f1})

    if generation % 5 == 0 or generation == NUM_GENERATIONS - 1:
        print(f"Generation {generation + 1}: Best Fitness = {best_f1:.7f}")

iteration_df = pd.DataFrame(iteration_data)
import ace_tools as tools
tools.display_dataframe_to_user(name="Iteration and F1 Scores", dataframe=iteration_df)

best_individual = max(population, key=lambda ind: get_fitness(generate_prompt(ind), "Extract the interaction between {gene1} and {gene2}.", temperature=0.9))
print(f"\nBest prompt of the final generation: {generate_prompt(best_individual)}")

In [ ]:
import pandas as pd
import random
import numpy as np
import openai
from sklearn.metrics import f1_score

# OpenAI API key
OPENAI_API_KEY = "your_api_key_here"

# File paths
roles_path = "/content/drive/MyDrive/roles.csv"
aims_path = "/content/drive/MyDrive/aims.csv"
instructions_path = "/content/drive/MyDrive/instructions.csv"
descriptions_path = "/content/drive/MyDrive/descriptions.csv"

# Load CSV files
roles_df = pd.read_csv(roles_path)
aims_df = pd.read_csv(aims_path)
instructions_df = pd.read_csv(instructions_path)
descriptions_df = pd.read_csv(descriptions_path)

# Ensure all dataframes have the same number of rows
max_length = max(len(roles_df), len(aims_df), len(instructions_df), len(descriptions_df))
roles_df = roles_df.sample(n=max_length, replace=True).reset_index(drop=True)
aims_df = aims_df.sample(n=max_length, replace=True).reset_index(drop=True)
instructions_df = instructions_df.sample(n=max_length, replace=True).reset_index(drop=True)
descriptions_df = descriptions_df.sample(n=max_length, replace=True).reset_index(drop=True)

# Convert to lists
roles = roles_df.iloc[:, 0].tolist()
aims = aims_df.iloc[:, 0].tolist()
instruction = instructions_df.iloc[:, 0].tolist()
descriptions = descriptions_df.iloc[:, 0].tolist()

def ask_question(prompt, question, temperature=0.9):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": prompt}, {"role": "user", "content": question}],
            temperature=temperature
        )
        return response["choices"][0]["message"]["content"].strip().lower()
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

def get_fitness(prompt, question, temperature=0.9, training_path="/content/drive/MyDrive/embedding relation/training.xlsx"):
    training_df = pd.read_excel(training_path).sample(frac=1).reset_index(drop=True)
    answers = []
    for _, row in training_df.iterrows():
        gpt4o_answer = ask_question(prompt, question.format(gene1=row['starter'].upper(), gene2=row['receiver'].upper()), temperature)
        answers.append({'starter': row['starter'], 'receiver': row['receiver'], 'relation': row['relation_name'], 'predict_relation': gpt4o_answer})

    answer_df = pd.DataFrame(answers)
    overall_f1, _ = calculate_metrics(answer_df)
    return overall_f1

def select_parents(population, fitness_values):
    probabilities = [fitness / sum(fitness_values) for fitness in fitness_values]
    return random.choices(population, weights=probabilities, k=int(POPULATION_SIZE * SELECTION_PROPORTION))

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1) - 1)
    return (parent1[:point] + parent2[point:], parent2[:point] + parent1[point:])

def mutate(individual):
    mutated = list(individual)
    index = random.randint(0, len(mutated) - 1)
    options = [roles, aims, descriptions, instruction][index]
    mutated[index] = random.choice(options)
    return tuple(mutated)

def generate_prompt(individual):
    return f"As a {individual[0]}, {individual[1]}. {individual[2]} {individual[3]}."

# Genetic algorithm parameters
POPULATION_SIZE = 9
NUM_GENERATIONS = 30
MUTATION_RATE = 0.3
SELECTION_PROPORTION = 0.5

population = [(random.choice(roles), random.choice(aims), random.choice(descriptions), random.choice(instruction)) for _ in range(POPULATION_SIZE)]

iteration_data = []

for generation in range(NUM_GENERATIONS):
    fitness_values = [get_fitness(generate_prompt(ind), "Extract the interaction between {gene1} and {gene2}.") for ind in population]
    new_population = select_parents(population, fitness_values)

    while len(new_population) < POPULATION_SIZE:
        p1, p2 = random.sample(new_population, 2)
        c1, c2 = crossover(p1, p2)
        if random.random() < MUTATION_RATE:
            c1 = mutate(c1)
        if random.random() < MUTATION_RATE:
            c2 = mutate(c2)
        new_population.extend([c1, c2])

    population = sorted(new_population[:POPULATION_SIZE], key=lambda ind: get_fitness(generate_prompt(ind), "Extract the interaction between {gene1} and {gene2}.", temperature=0.9), reverse=True)

    best_f1 = max(fitness_values)
    iteration_data.append({"Iteration": generation + 1, "Best F1 Score": best_f1})

    if generation % 5 == 0 or generation == NUM_GENERATIONS - 1:
        print(f"Generation {generation + 1}: Best Fitness = {best_f1:.7f}")

iteration_df = pd.DataFrame(iteration_data)
import ace_tools as tools
tools.display_dataframe_to_user(name="Iteration and F1 Scores", dataframe=iteration_df)

best_individual = max(population, key=lambda ind: get_fitness(generate_prompt(ind), "Extract the interaction between {gene1} and {gene2}.", temperature=0.9))
print(f"\nBest prompt of the final generation: {generate_prompt(best_individual)}")


In [ ]:
import pandas as pd

# File paths
roles_path = "/content/drive/MyDrive/roles.csv"
aims_path = "/content/drive/MyDrive/aims.csv"
instructions_path = "/content/drive/MyDrive/instructions.csv"
descriptions_path = "/content/drive/MyDrive/descriptions.csv"

# Load CSV files
roles_df = pd.read_csv(roles_path)
aims_df = pd.read_csv(aims_path)
instructions_df = pd.read_csv(instructions_path)
descriptions_df = pd.read_csv(descriptions_path)

# Ensure all dataframes have at least the same number of rows by repeating values if necessary
max_length = max(len(roles_df), len(aims_df), len(instructions_df), len(descriptions_df))

roles_df = roles_df.sample(n=max_length, replace=True).reset_index(drop=True)
aims_df = aims_df.sample(n=max_length, replace=True).reset_index(drop=True)
instructions_df = instructions_df.sample(n=max_length, replace=True).reset_index(drop=True)
descriptions_df = descriptions_df.sample(n=max_length, replace=True).reset_index(drop=True)

# Combine all data into a single DataFrame
merged_df = pd.DataFrame({
    "Roles": roles_df.iloc[:, 0],
    "Aims": aims_df.iloc[:, 0],
    "Instructions": instructions_df.iloc[:, 0],
    "Descriptions": descriptions_df.iloc[:, 0]
})

# Save as CSV file
merged_csv_path = "/content/drive/MyDrive/marge-all.csv"
merged_df.to_csv(merged_csv_path, index=False)

# Display the CSV file to the user
import ace_tools as tools
tools.display_dataframe_to_user(name="Merged Data (marge-all)", dataframe=merged_df)